In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
!pip install -q keras

In [0]:
!pip install -q pydrive

     |████████████████████████████████| 993kB 3.4MB/s 


In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding,Bidirectional, RepeatVector, Concatenate, Activation, Dot, Lambda
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.backend as K

In [0]:
if len(K.tensorflow_backend._get_available_gpus()) > 0:
  print("GPU Available !!!!!")
  from keras.layers import CuDNNLSTM as LSTM
  from keras.layers import CuDNNGRU as GRU

GPU Available !!!!!


In [0]:
from sklearn.model_selection import train_test_split

def getData(language='ben',N_MAX=1000000):
    train_input_texts = []  # sentence in original language
    train_target_texts = []  # sentence in target language
    train_target_texts_inputs = []  # sentence in target language offset by 1

    test_input_texts = []  # sentence in original language
    test_target_texts = []  # sentence in target language
    test_target_texts_inputs = []  # sentence in target language offset by 1

    k=0

    input_list=[]
    translation_list=[]
    for line in open('/content/drive/My Drive/Machine Learning/Udemy/Deep Learning Advanced NLP and RNNs/Attention/Neural Machine Translation/'
                     +str(language)+'.txt', encoding='utf-8'):

        k += 1
        if k > N_MAX:
            break


        if '\t' not in line:
            continue

        input_text, translation = line.rstrip().split('\t')

        input_list.append(input_text)
        translation_list.append(translation)
        
    print('Total data got: '+str(k))

    train_input_list,test_input_list,train_translation_list,test_translation_list=train_test_split(input_list,translation_list,
                                                                                            test_size=0.2,random_state=42)
    #print(len(train_input_list))

    for input_text,translation in zip(train_input_list,train_translation_list):

        # make the target input and output
        # recall we'll be using teacher forcing
        target_text = translation + ' <eos>'
        target_text_input = '<sos> ' + translation

        train_input_texts.append(input_text)
        train_target_texts.append(target_text)
        train_target_texts_inputs.append(target_text_input)

    test_input_texts=list(test_input_list)
    test_target_texts=list(test_translation_list)

    print("num train samples:", len(train_input_texts))
    print(train_target_texts[0])
    

    return train_input_texts,train_target_texts,train_target_texts_inputs,test_input_texts,test_target_texts

In [0]:
MAX_TOKENS=20000
EMBEDDING_SIZE=100


input_texts,target_texts,target_texts_inputs,test_input_texts,test_target_texts=getData(language='ben',N_MAX=10000)
N=len(input_texts)

Total data got: 4387
num train samples: 3509
টম মেরিকে বললো যে ও নিজেকে হত্যা করতে চলেছিলো, কিন্ত তা করার মতো সাহস ছিলো না। <eos>


In [0]:
input_tokenizer=Tokenizer(num_words=MAX_TOKENS)
input_tokenizer.fit_on_texts(input_texts)
input_sequences=input_tokenizer.texts_to_sequences(input_texts)
MAX_LEN_INPUT=max(len(i)for i in input_sequences)
encoder_inputs=pad_sequences(input_sequences,maxlen=MAX_LEN_INPUT)

output_tokenizer=Tokenizer(num_words=MAX_TOKENS,filters='')
output_tokenizer.fit_on_texts(['<sos>']+target_texts)
#output_tokenizer.fit_on_texts(target_texts_inputs+target_texts)
target_sequences=output_tokenizer.texts_to_sequences(target_texts)
MAX_LEN_OUTPUT=max(len(i)for i in target_sequences)
decoder_targets=pad_sequences(target_sequences,maxlen=MAX_LEN_OUTPUT,padding='post')

target_input_sequences=output_tokenizer.texts_to_sequences(target_texts_inputs)
decoder_inputs=pad_sequences(target_input_sequences,maxlen=MAX_LEN_OUTPUT,padding='post')

input_vocab_size=len(input_tokenizer.word_index)+1
output_vocab_size=len(output_tokenizer.word_index)+1

In [0]:
f = open('/content/drive/My Drive/Machine Learning/Fake News/Evaluation/Evaluation/glove.6B.100d.txt',encoding='utf-8')
embeddings={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((input_vocab_size, EMBEDDING_SIZE))
for word, i in input_tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Total 400000 word vectors.


In [0]:
target_one_hot_vector=np.zeros((len(target_texts),MAX_LEN_OUTPUT,output_vocab_size),dtype='float32')

for sample,text in enumerate(decoder_targets):
    for word_position,word_index in enumerate(text):
        target_one_hot_vector[sample,word_position,word_index]=1

In [0]:
test_input_sequences=input_tokenizer.texts_to_sequences(test_input_texts)
test_encoder_inputs=pad_sequences(test_input_sequences,maxlen=MAX_LEN_INPUT)


test_target_sequences=output_tokenizer.texts_to_sequences(test_target_texts)
test_decoder_targets=pad_sequences(test_target_sequences,maxlen=MAX_LEN_OUTPUT,padding='post')

In [0]:
test_target_one_hot_vector=np.zeros(shape=(len(test_target_texts),MAX_LEN_OUTPUT,output_vocab_size),dtype='float32')

for sample,text in enumerate(test_decoder_targets):
    for word_position,word_index in enumerate(text):
        test_target_one_hot_vector[sample,word_position,word_index]=1

In [0]:
max_input_length=MAX_LEN_INPUT
max_output_length=MAX_LEN_OUTPUT
embedding_size=EMBEDDING_SIZE


NN_dim_1=256
NN_dim_2=10
NN_dim_3=256

output_words=output_vocab_size

In [0]:
def time_distributed_softmax(x):
    # x = N x Time x Dimension
    normalize_x=x-K.max(x,axis=1,keepdims=True) # max_x[i,j] = max(x[i,:,j]) as axis = 1
    exp_x=K.exp(normalize_x)
    softmax=exp_x / K.sum(exp_x,axis=1,keepdims=True) # sum_x[i,j] = sum(x[i,:,j]) as axis = 1
    return softmax

In [0]:
def stack(x):
    x = K.stack(x)
    x = K.permute_dimensions(x, pattern=(1, 0, 2))
    return x

In [0]:
encoder_input=Input(shape=(max_input_length,),)
encoder_embedding = Embedding(input_dim=embedding_matrix.shape[0], output_dim=embedding_size,
                              weights=[embedding_matrix],
                              input_length=max_input_length)


encoder_embedding_layer=encoder_embedding(encoder_input)
layer1=Bidirectional(LSTM(NN_dim_1,return_sequences=True))
encoder_output=layer1(encoder_embedding_layer)

In [0]:
repeat_decoder_state=RepeatVector(max_input_length)

enc_dec_state_concat=Concatenate(axis=-1)

dense1=Dense(NN_dim_2,activation='tanh')

dense2 = Dense(1, activation=time_distributed_softmax)

h_alphas_dot=Dot(axes=1)

In [0]:
def attention_for_one_step(encoder_states_h,decoder_state_s_t_prev):
  #######################################   ATTENTION LAYER for 1 Step #########################################


  # it is one step, which will repeated in the decoder max_output_length times
  # will return context vector for one time step

  # before decoder_state_s_t_prev dim = NN_dim_3
  decoder_state_s_t_prev=repeat_decoder_state(decoder_state_s_t_prev)
  # after decoder_state_s_t_prev dim = (max_input_length,NN_dim_3)
  # we need to repeat it to concat with all encoder_states_h

  # encoder_states_h dim = (max_input_length,NN_dim_1 *2) as Bidirectional
  
  h_s_t_prev_concat=enc_dec_state_concat([encoder_states_h,decoder_state_s_t_prev])
  # h_s_t_prev_concat dim = (max_input_length,NN_dim_1 *2 + NN_dim_3)
  
  attn_layer1=dense1(h_s_t_prev_concat)

  # Now we get alphas with special over time activation softmax
  alphas=dense2(attn_layer1)

  context_vector=h_alphas_dot([alphas,encoder_states_h]) # context vector for one time step. phew!

  return context_vector

In [0]:
# max_output_length is which we define in Padding
decoder_target_input=Input(shape=(max_output_length,),)

decoder_embedding = Embedding(input_dim=output_words, output_dim=embedding_size)


decoder_embedding_layer=decoder_embedding(decoder_target_input)


layer1=LSTM(NN_dim_3,return_state=True)
layer2=Dense(output_words,activation='softmax')

s0=Input(shape=(NN_dim_3,),)
c0 = Input(shape=(NN_dim_3,), )

s=s0
c=c0

outputs=[]

for t in range(max_output_length):
    decoder_state_s_t_prev=s
    
    #######################################   ATTENTION LAYER for 1 Step #########################################
    
    
    context_vector=attention_for_one_step(encoder_output,decoder_state_s_t_prev)
    
    #######################################   ATTENTION LAYER for 1 Step #########################################

    target_input_t=Lambda(lambda x: x[:,t:t+1])
    teacher_forcing=target_input_t(decoder_embedding_layer)

    context_teacher_concat=Concatenate(axis=2)
    decoder_layer1_input=context_teacher_concat([context_vector,teacher_forcing])

    decoder_layer1,s,c=layer1(decoder_layer1_input,initial_state=[s,c])
    decoder_output=layer2(decoder_layer1)

    outputs.append(decoder_output)

In [0]:
#outputs shape =  max_output_length x N x Dimension. We need to convert it to N x max_output_length x Dimension
change_shape=Lambda(stack)
outputs=change_shape(outputs)

In [0]:
initial_s=Input(shape=(NN_dim_3,))
initial_c = Input(shape=(NN_dim_3,))

train_model=Model(inputs=[encoder_input,decoder_target_input,s0,c0],outputs=outputs)

In [0]:
from keras import callbacks

def callback(model_name,tf_log_dir_name='./tf-log/',patience_lr=10,):
    cb = []
    """
    Tensorboard log callback
    """
    #tb = callbacks.TensorBoard(log_dir=tf_log_dir_name, histogram_freq=0)
    #cb.append(tb)

    """
    Model-Checkpoint
    """
    #m = callbacks.ModelCheckpoint(filepath=model_name,monitor='val_loss',mode='auto',save_best_only=True)
    #cb.append(m)

    """
    Reduce Learning Rate
    """
    reduce_lr_loss = callbacks.ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=1, min_delta=1e-4, mode='min')
    cb.append(reduce_lr_loss)

    """
    Early Stopping callback
    """
    # Uncomment for usage
    early_stop = callbacks.EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')
    cb.append(early_stop)



    return cb

In [0]:
train_model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])
#train_model.summary()

s_c = np.zeros((N, NN_dim_3))  # initial [s, c]

cb=callback(model_name='/content/drive/My Drive/Machine Learning/Udemy/Deep Learning Advanced NLP and RNNs/Attention/Neural Machine Translation/Model_NML_1.h5')


history=train_model.fit([encoder_inputs, decoder_inputs, s_c, s_c], target_one_hot_vector,
            epochs=50, batch_size=64, validation_split=0.2, shuffle=True, callbacks=cb)

Train on 2807 samples, validate on 702 samples
Epoch 1/50
2807/2807 [==============================] - 17s 6ms/step - loss: 2.5115 - acc: 0.6902 - val_loss: 2.0746 - val_acc: 0.7096
Epoch 2/50
2807/2807 [==============================] - 3s 955us/step - loss: 1.8395 - acc: 0.7284 - val_loss: 1.7900 - val_acc: 0.7492
Epoch 3/50
2807/2807 [==============================] - 3s 975us/step - loss: 1.6798 - acc: 0.7611 - val_loss: 1.7498 - val_acc: 0.7688
Epoch 4/50
2807/2807 [==============================] - 3s 961us/step - loss: 1.5891 - acc: 0.7706 - val_loss: 1.6969 - val_acc: 0.7692
Epoch 5/50
2807/2807 [==============================] - 3s 981us/step - loss: 1.5101 - acc: 0.7725 - val_loss: 1.6392 - val_acc: 0.7726
Epoch 6/50
2807/2807 [==============================] - 3s 1ms/step - loss: 1.4290 - acc: 0.7772 - val_loss: 1.5917 - val_acc: 0.7794
Epoch 7/50
2807/2807 [==============================] - 3s 1ms/step - loss: 1.3482 - acc: 0.7828 - val_loss: 1.5516 - val_acc: 0.7825
Epoch 

In [0]:

test_encoder_model=Model(encoder_input,encoder_output)


'''#########################  Start Input from the place, at where the graph is changing. Here we do not have the previous decoder embedding
and so, teacher forcing is going to be changed. 
For this reason we need to build a seperate encoder model because previous encoder was connected to teacher forcing. ############ ''' 

test_decoder_input=Input(shape=(1,),)
decoder_single_input=decoder_embedding(test_decoder_input)

encoder2decoder_input=Input(shape=(max_input_length,2*NN_dim_1,),)

context_vector=attention_for_one_step(encoder2decoder_input,s0)
decoder_layer1_input=context_teacher_concat([context_vector,decoder_single_input])

decoder_layer1,s,c=layer1(decoder_layer1_input,initial_state=[s0,c0])
outputs=layer2(decoder_layer1)


test_decoder_model=Model(inputs=[test_decoder_input,encoder2decoder_input,s0,c0],outputs=[outputs,s,c])


In [0]:
input_idx2word={i:w for w,i in input_tokenizer.word_index.items()}
output_idx2word={i:w for w,i in output_tokenizer.word_index.items()}


def decode_sequence(input_sequence):
  encoder2decoder=test_encoder_model.predict(input_sequence)
  
  target_input=np.zeros(shape=(1,1))
  target_input[0,0]=output_tokenizer.word_index['<sos>']
  #target_input[0,0]=7
  
  eos=output_tokenizer.word_index['<eos>']
  
  s= np.zeros((1, NN_dim_3))
  c = np.zeros((1, NN_dim_3))
  
  outputs=[]
  
  for i in range(max_output_length):

    o,s,c=test_decoder_model.predict([target_input,encoder2decoder,s,c])
    
    #print(o.shape)
    word_idx=np.argmax(o.flatten())
    #print(word_idx)
    
    if(eos==word_idx):
      break
    
    if(word_idx>0):
      outputs.append(output_idx2word[word_idx])
    target_input[0,0]=word_idx
   
  return ' '.join(outputs)
    
    
  

In [0]:
for i in range(len(test_input_texts)):
  print('Input: ' + test_input_texts[i])
  print('Output: ' + test_target_texts[i])
  print('Predicted Output: '+ decode_sequence(test_encoder_inputs[i:i+1]))

Input: How could they forget us?
Output: তারা আমাদের কী করে ভুলে যেতে পারে?
Predicted Output: তাঁরা আমাদের কী ভুলে যেতে পারেন?
Input: At what time does it close?
Output: কটার সময এটা বন্ধ হয়?
Predicted Output: কটার কটার সময় বন্ধ হয়?
Input: You should know better.
Output: আপনিই ভালো করে জানেন।
Predicted Output: তোমরাই ভালো করে জানো।
Input: They screamed.
Output: তাঁরা চিৎকার করলেন।
Predicted Output: ওরা চেঁচালো।
Input: My head aches.
Output: আমার মাথা ব্যাথা করছে।
Predicted Output: আমার বাবা ধরো।
Input: Grab Tom.
Output: টমকে ধরুন।
Predicted Output: টমকে ধরো।
Input: I shouldn't have logged off.
Output: আমার লগ আউট করা উচিৎ হয়নি।
Predicted Output: আমি তিনটে গেছিলাম।
Input: What is the distance from here to the station?
Output: এখান থেকে স্টেশনের দূরত্ব কতটা?
Predicted Output: ট্রেনটা কটার সময় ছাড়ে?
Input: Tom is taller than Mary.
Output: টম মেরির থেকে লম্বা।
Predicted Output: টম মেরির থেকে রয়েছে।
Input: Do you want to eat now or later?
Output: তুমি কি এখন খেতে চাও না পরে?
Predicted Outpu

**##########################################    Testing Zone Starts from Here   ####################################**

In [0]:
print(output_idx2word[20])

টমকে


In [0]:
print('Input: ' + input_texts[0])
print('Output: ' + target_texts[0])
print(decode_sequence(encoder_inputs[0:1]))

Input: Throw it to Tom.
Output: Tíraselo a Tom. <eos>



In [0]:
##### build the model #####

# Set up the encoder - simple!
encoder_inputs_placeholder = Input(shape=(max_input_length,))
x = encoder_embedding(encoder_inputs_placeholder)
encoder = Bidirectional(LSTM(
  NN_dim_1,
  return_sequences=True,
  # dropout=0.5 # dropout not available on gpu
))
encoder_outputs = encoder(x)


# Set up the decoder - not so simple
decoder_inputs_placeholder = Input(shape=(max_output_length,))

# this word embedding will not use pre-trained vectors
# although you could
decoder_embedding = Embedding(output_words, embedding_size)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)




######### Attention #########
# Attention layers need to be global because
# they will be repeated Ty times at the decoder
attn_repeat_layer = RepeatVector(max_input_length)
attn_concat_layer = Concatenate(axis=-1)
attn_dense1 = Dense(10, activation='tanh')
attn_dense2 = Dense(1, activation=time_distributed_softmax)
attn_dot = Dot(axes=1) # to perform the weighted sum of alpha[t] * h[t]

def one_step_attention(h, st_1):
  # h = h(1), ..., h(Tx), shape = (Tx, LATENT_DIM * 2)
  # st_1 = s(t-1), shape = (LATENT_DIM_DECODER,)
 
  # copy s(t-1) Tx times
  # now shape = (Tx, LATENT_DIM_DECODER)
  st_1 = attn_repeat_layer(st_1)

  # Concatenate all h(t)'s with s(t-1)
  # Now of shape (Tx, LATENT_DIM_DECODER + LATENT_DIM * 2)
  x = attn_concat_layer([h, st_1])

  # Neural net first layer
  x = attn_dense1(x)

  # Neural net second layer with special softmax over time
  alphas = attn_dense2(x)

  # "Dot" the alphas and the h's
  # Remember a.dot(b) = sum over a[t] * b[t]
  context = attn_dot([alphas, h])

  return context


# define the rest of the decoder (after attention)
decoder_lstm = LSTM(NN_dim_3, return_state=True)
decoder_dense = Dense(output_words, activation='softmax')

initial_s = Input(shape=(NN_dim_3,), name='s0')
initial_c = Input(shape=(NN_dim_3,), name='c0')
context_last_word_concat_layer = Concatenate(axis=2)


# Unlike previous seq2seq, we cannot get the output
# all in one step
# Instead we need to do Ty steps
# And in each of those steps, we need to consider
# all Tx h's

# s, c will be re-assigned in each iteration of the loop
s = initial_s
c = initial_c

# collect outputs in a list at first
outputs = []
for t in range(max_output_length): # Ty times
  # get the context using attention
  context = one_step_attention(encoder_outputs, s)
  
  #print(encoder_outputs.shape)

  # we need a different layer for each time step
  selector = Lambda(lambda x: x[:, t:t+1])
  xt = selector(decoder_inputs_x)
  
  # combine 
  decoder_lstm_input = context_last_word_concat_layer([context, xt])

  # pass the combined [context, last word] into the LSTM
  # along with [s, c]
  # get the new [s, c] and output
  o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[s, c])

  # final dense layer to get next word prediction
  decoder_outputs = decoder_dense(o)
  outputs.append(decoder_outputs)


# 'outputs' is now a list of length Ty
# each element is of shape (batch size, output vocab size)
# therefore if we simply stack all the outputs into 1 tensor
# it would be of shape T x N x D
# we would like it to be of shape N x T x D

def stack_and_transpose(x):
  # x is a list of length T, each element is a batch_size x output_vocab_size tensor
  x = K.stack(x) # is now T x batch_size x output_vocab_size tensor
  x = K.permute_dimensions(x, pattern=(1, 0, 2)) # is now batch_size x T x output_vocab_size
  return x

# make it a layer
stacker = Lambda(stack_and_transpose)
outputs = stacker(outputs)

# create the model
model = Model(
  inputs=[
    encoder_inputs_placeholder,
    decoder_inputs_placeholder,
    initial_s, 
    initial_c,
  ],
  outputs=outputs
)

In [0]:
##### Make predictions #####
# As with the poetry example, we need to create another model
# that can take in the RNN state and previous word as input
# and accept a T=1 sequence.

# The encoder will be stand-alone
# From this we will get our initial decoder hidden state
# i.e. h(1), ..., h(Tx)
encoder_model = Model(encoder_inputs_placeholder, encoder_outputs)

# next we define a T=1 decoder model
encoder_outputs_as_input = Input(shape=(max_input_length, NN_dim_1 * 2,))
decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

# no need to loop over attention steps this time because there is only one step
context = one_step_attention(encoder_outputs_as_input, initial_s)

# combine context with last word
decoder_lstm_input = context_last_word_concat_layer([context, decoder_inputs_single_x])

# lstm and final dense
o, s, c = decoder_lstm(decoder_lstm_input, initial_state=[initial_s, initial_c])
decoder_outputs = decoder_dense(o)


# note: we don't really need the final stack and tranpose
# because there's only 1 output
# it is already of size N x D
# no need to make it 1 x N x D --> N x 1 x D



# create the model object
decoder_model = Model(
  inputs=[
    decoder_inputs_single,
    encoder_outputs_as_input,
    initial_s, 
    initial_c
  ],
  outputs=[decoder_outputs, s, c]
)



# map indexes back into real words
# so we can view the results
idx2word_eng = {v:k for k, v in input_tokenizer.word_index.items()}
idx2word_trans = {v:k for k, v in output_tokenizer.word_index.items()}





def decode_sequence(input_seq):
  # Encode the input as state vectors.
  enc_out = encoder_model.predict(input_seq)

  # Generate empty target sequence of length 1.
  target_seq = np.zeros((1, 1))
  
  # Populate the first character of target sequence with the start character.
  # NOTE: tokenizer lower-cases all words
  target_seq[0, 0] = output_tokenizer.word_index['<sos>']

  # if we get this we break
  eos = output_tokenizer.word_index['<eos>']


  # [s, c] will be updated in each loop iteration
  s = np.zeros((1, NN_dim_3))
  c = np.zeros((1, NN_dim_3))


  # Create the translation
  output_sentence = []
  for _ in range(max_output_length):
    o, s, c = decoder_model.predict([target_seq, enc_out, s, c])
        

    # Get next word
    #print(o.flatten())
    idx = np.argmax(o.flatten())
    #print(idx)

    # End sentence of EOS
    if eos == idx:
      break

    word = ''
    if idx > 0:
      word = idx2word_trans[idx]
      output_sentence.append(word)

    # Update the decoder input
    # which is just the word just generated
    target_seq[0, 0] = idx

  return ' '.join(output_sentence)

In [0]:
# compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

# train the model
z = np.zeros((len(encoder_inputs), NN_dim_3)) # initial [s, c]
r = model.fit(
  [encoder_inputs, decoder_inputs, z, z], target_one_hot_vector,
  batch_size=64,
  epochs=10,
  validation_split=0.2
)